In [4]:
import cv2
import numpy as np
from sklearn.metrics import pairwise
import tkinter as tk
import tkinter.messagebox


print("openCV :")
print(cv2.__version__)

openCV :
4.1.2


In [5]:
root = tk.Tk()
root.title("Lets_Count_them_fingers")
root.geometry("500x500")
root.resizable(0,0)
root.configure(bg='black')


In [6]:
def Lets_Count_them_fingers():
    
    global bgd
    bgd = None

    # halfway point between 0 and 1 of accumulated weight
    accumulated_weight = 0.5

    
    # ROI for our hand.
    roi_top = 20
    roi_bottom = 300
    roi_right = 300
    roi_left = 600
    
    
    #####################################################################################################################
   

    # Given a frame and a previous accumulated weight, computed the weighted average of the image passed in.
    # compute weighted average, accumulate it and update the background
    
    def calc_accum_avg(frame, accumulated_weight):
 
        global bgd
    
        if bgd is None:
            bgd = frame.copy().astype("float")
            return None

        cv2.accumulateWeighted(frame, bgd, accumulated_weight)
   

    ######################################################################################################################
  

     # Calculates the Absolute Differentce between the backgroud and the passed in frame
     # Return both the hand segment and the thresholded hand image
    
    def segment(frame, threshold=25):
        
        global bgd
    
        diff = cv2.absdiff(bgd.astype("uint8"), frame)
    
        ret , thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)

        contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if len(contours) == 0:
            return None
        else:
        
            # the largest external contour is our hand = segment
            hand_segment = max(contours, key=cv2.contourArea)
        
            return (thresholded, hand_segment)
   

    #####################################################################################################################
    
     
    def count_fingers(thresholded, hand_segment):
    
        # Calculated the convex hull of the hand segment 
        conv_hull = cv2.convexHull(hand_segment)
    
    
        # Now the convex hull will have at least 4 most outward points, on the top, bottom, left, and right.
        top    = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
        bottom = tuple(conv_hull[conv_hull[:, :, 1].argmax()][0])
        left   = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])
        right  = tuple(conv_hull[conv_hull[:, :, 0].argmax()][0])
        
        # center pts
        cX = (left[0] + right[0]) // 2
        cY = (top[1] + bottom[1]) // 2

        
        # find the maximum euclidean distance between the center of the palm
        # and the most extreme points of the convex hull
        distance = pairwise.euclidean_distances([(cX, cY)], Y=[left, right, top, bottom])[0]
    
    
        # Grab the largest distance
        max_distance = distance.max()
        
    
        # Create a circle with 80% radius of the max euclidean distance
        radius = int(0.8 * max_distance)
        circumference = (2 * np.pi * radius)

        
        # Now grab an ROI of only that circle
        circular_roi = np.zeros(thresholded.shape[:2], dtype="uint8")
    
    
        # draw the circular ROI
        cv2.circle(circular_roi, (cX, cY), radius, 255, 10)
        circular_roi = cv2.bitwise_and(thresholded, thresholded, mask=circular_roi)

        
        # apply contours
        contours, hierarchy = cv2.findContours(circular_roi.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  

        #count our fingers
        count = 0
        

        # loop through the contours for more fingers.
        for cnt in contours:
        
            (x, y, w, h) = cv2.boundingRect(cnt)
        
            # 1. make sure the Contour region is not the bottom of hand area 
            # [i.e TO MAKE SURE IT IS NOT R WRIST ]
            out_of_wrist = ((cY + (cY * 0.25)) > (y + h))
        
        
            # 2. limit the  points along the contour such that they do not exceed 25% of the circumference of the circular ROI 
            # [ TO AVOID FALLING OFF THE FINGERS  ]
            limit_points = ((circumference * 0.25) > cnt.shape[0])
        
        
            if  out_of_wrist and limit_points:
                count += 1

        return count


    ####################################################################################################################
    
    #initiate your cam
    cam = cv2.VideoCapture(0)
    num_frames = 0

    while True:
        ret, frame = cam.read()
        frame = cv2.flip(frame, 1)
        frame_copy = frame.copy()

        # roi are
        roi = frame[roi_top:roi_bottom, roi_right:roi_left]

        
        #exclusive finger scanner
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        
        # detect the hand for the first 30 frames to avoid noise (obstructions, other objects)
        if num_frames < 60:
            calc_accum_avg(gray, accumulated_weight)
            if num_frames <= 59:
                cv2.putText(frame_copy, "WAIT! GETTING BACKGROUND AVG.", (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
                cv2.imshow("Finger Count",frame_copy)
            
        else:
            # segment the hand.
            hand = segment(gray)

            
            # did we detect a hand ?
            if hand is not None:
            
                thresholded, hand_segment = hand
            
                cv2.drawContours(frame_copy, [hand_segment + (roi_right, roi_top)], -1, (255, 0, 0),1)

                fingers = count_fingers(thresholded, hand_segment)

                cv2.putText(frame_copy, str(fingers), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

                cv2.imshow("Thesholded", thresholded)

        cv2.rectangle(frame_copy, (roi_left, roi_top), (roi_right, roi_bottom), (0,0,255), 5)

        num_frames += 1

        cv2.imshow("Finger Count", frame_copy)

        
        # wait for a key press
        k = cv2.waitKey(1) & 0xFF

        # hit ESCAPE key to exit 
        if k == 27:
            break

    #succesfully end the display
    cam.release()
    cv2.destroyAllWindows()
    
    
    #####################################################################################################################

    f = tk.Tk()
    f.geometry("500x500")
    f.title(" Lets_count_them_Fingers")
    tkinter.messagebox.showinfo(" (^_^) ")
    f.mainloop()

b=tk.Button(root,text="Click Here !",height=5,width=15,font=('cooperblack',10,'bold'),
            bg='MediumPurple3',fg='white',command=Lets_Count_them_fingers)

b.place(relx=0.5,rely=0.5,anchor='c')
tk.mainloop()















